In [1]:
import pathlib
from lxml import etree
import random
import requests
import json
import time
from bs4 import BeautifulSoup
import re
import csv
import os

In [2]:
# os.getcwd()
os.chdir('C:/users/jjett2/desktop/SynthBio/')
os.getcwd()

'C:\\users\\jjett2\\desktop\\SynthBio'

In [3]:
#jsonsource = pathlib.Path('json-ner-annotation-20210205.tar/json-ner-annotation-20210205/')

#jfiles = list(jsonsource.glob('*.json'))

#nermentionsdata = {}
#for f in jfiles:
#    data = json.loads(f.read_text(encoding = 'utf-8'))
#    pubnum = data['pub_num']
#    nermentions = []
#    for paragraph in data['body']:
#        if 'ner_annotations' in paragraph:
#            for mention in paragraph['ner_annotations']:
#                nermentions.append(mention['ner_mention'])
#    nermentionsdata[pubnum] = list(set(nermentions))
    
#indemediatetarget = 'nermentions.json'
    
# use this to write out the structure
# if you edit the file, comment this out so you don't overwrite
#with open(indemediatetarget, 'w') as fout:
#    json.dump(nermentionsdata, fout)
#temporarily done with this step.

In [4]:
#!dir

In [5]:
# # use this to read in a copy of the file (should you want to edit it)
# execute separately

#def load_ner_mentions(pubnum):
#    with open(indemediatetarget, 'r') as fin:
#        nermentionsdata = json.load(fin)
#        if pubnum in nermentionsdata:
#            return nermentionsdata[pubnum]
#        else:
#            return []


# if there are additional cleaned datasets, need to add the code to grab their data here
#note that these use specific directory paths and need to be adjusted to the local compute environment
with open(r"C:\Users\jjett2\Desktop\SynthBio\NER Data Products\Chemical Dataset\chebiValues.json", "r", encoding="utf-8") as infile:
    chemicals = json.load(infile)

with open(r"C:\Users\jjett2\Desktop\SynthBio\NER Data Products\Species Dataset\ncbiValues.json", "r", encoding="utf-8") as infile:
    species = json.load(infile)
    
with open(r"C:\Users\jjett2\Desktop\SynthBio\NER Data Products\Species Dataset\ncbiStrainValues.json", "r", encoding="utf-8") as infile:
    strains = json.load(infile)  

        

In [6]:
#chemicals.keys()

In [7]:
xmlfiles = list(pathlib.Path('Data/').glob('**/*.xml'))

In [8]:
originalfiles = [f for f in xmlfiles if not f.stem.endswith('-RDF')]
# for use when you are rerunning after creating these

In [9]:
# tool for randomly choosing one of the files and looking at it
# this is useful for testing purposes
rando = random.choice(originalfiles)
# print(rando)
#print(chemicals)

In [10]:
rando

WindowsPath('Data/Data/sb6b00123/sb6b00123.xml')

In [11]:
#writes out the RDF/XML file
def create_new_xml(json_record):
#    article_id = pathlib.Path(str(path.stem))
    xmlout = """<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:prov="http://www.w3.org/ns/prov#" xmlns:sbol="http://sbols.org/v2#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#dateTime/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:obo="http://purl.obolibrary.org/obo/">"""
    xmlout += '\n    <sbol:Collection rdf:about="https://dummy.org/' + json_record['sbol:displayId'] + '/1">'
#    xmlout += '\n    <sbol:Attachment rdf:about="https://dummy.org/' + json_record['doi'] + '/1">'
#   weaving an alternate for sbol:Collection -- making assumption that rdf:about entity ID remains the same
    xmlout += '\n        <sbol:persistentIdentity rdf:resource="https://dummy.org/'+ json_record['sbol:displayId'] + '"/>'
    xmlout += '\n       <sbol:displayId>' + json_record['sbol:displayId'] + '</sbol:displayId>'
    xmlout += '\n        <sbol:version>1</sbol:version>'
    xmlout += '\n        <prov:wasDerivedFrom rdf:resource="'+ json_record['doi_formatted'] + '"/>'
#    xmlout += '\n        <sbol:format rdf:resource="http://identifiers.org/edam/format_3508"/>'
    xmlout += '\n        <obo:OBI_0002110 rdf:resource="https://doi.org/' + json_record['doi'] + '"></obo:OBI_0002110>'
    if not json_record['pmid'] == "ERROR":
        xmlout += '\n        <obo:OBI_0001617>' + json_record['pmid'] + '</obo:OBI_0001617>'
    xmlout += '\n        <dcterms:type rdf:resource="http://purl.org/dc/dcmitype/Text"/>'
    xmlout += '\n        <dc:type>' + json_record['article_type'] +'</dc:type>'
    xmlout += '\n        <dcterms:isPartOf>' + json_record['jounal_title'] +'</dcterms:isPartOf>'
    xmlout += '\n        <dc:publisher>' + json_record['jounal_publisher'] + '</dc:publisher>'
    xmlout += '\n        <dcterms:isPartOf rdf:resource="http://pubs.acs.org/journal/asbcd6" />'
    xmlout += '\n        <dcterms:title>' + json_record['article-title'] + '</dcterms:title>'
    for each in json_record['authors']:
        xmlout += '\n        <dc:creator>' + each + '</dc:creator>'
    if not json_record['accepteddate'] == "ERROR":
        xmlout += '\n        <dcterms:created>' + json_record['accepteddate'] + '</dcterms:created>'
    xmlout += '\n        <dcterms:dateSubmitted>' + json_record['submitted_date'] + '</dcterms:dateSubmitted>'
    xmlout += '\n        <dcterms:dateCopyrighted>' + json_record['copyrightyear'] + '</dcterms:dateCopyrighted>'
    if not json_record['accepteddate'] == "ERROR":
        xmlout += '\n        <dcterms:dateAccepted>' + json_record['accepteddate'] + '</dcterms:dateAccepted>'
    if not json_record['copyrightstatement'] == "ERROR":
        xmlout += '\n        <dc:rights>' + json_record['copyrightstatement'] + '</dc:rights>'
    xmlout += '\n        <dcterms:rightsHolder>' + json_record['rightsholder'] + '</dcterms:rightsHolder>'
    if not json_record['abstract'] == "ERROR":
        xmlout += '\n        <dcterms:abstract>' + json_record['abstract'] + '</dcterms:abstract>'
        xmlout += '\n        <dcterms:description>' + json_record['abstract'] + '</dcterms:description>'
    for each in json_record['keywords']:
         xmlout += '\n        <dc:subject>' + each + '</dc:subject>'
#     for each in json_record['species']:
#         xmlout += '\n        <obo:IAO_0000136 rdf:about="' + each + '" />'
    if "chemicals" in json_record:
        print('adding chemical')
        for each in json_record['chemicals']:
            xmlout += '\n        <obo:IAO_0000136 rdf:about="' + each + '" /><!-- chemical -->'
            #print(each)
    if "species" in json_record:
        print('adding species')
        for each in json_record['species']:
            xmlout += '\n        <obo:IAO_0000136 rdf:about="' + each + '" /><!-- species -->'
            #print(each)
    if "strains" in json_record:
        print('adding strain')
        for each in json_record['strains']:
            xmlout += '\n        <obo:IAO_0000136 rdf:about="' + each + '" /><!-- strain -->'
            print(each)
    for each in json_record['refs']:
        if len(each) > 0:
            # ignore bad references
            xmlout += '\n        <dcterms:references>' + each + '</dcterms:references> '
    xmlout += '    </sbol:Collection>'
    xmlout += '</rdf:RDF>'
#    print(xmlout)
    soup = BeautifulSoup(xmlout, 'xml')
    tree = etree.fromstring(str(soup).encode('utf-8'))
#     tree = etree.fromstring(soup.prettify().encode('utf-8'))
    root = tree.xpath('.')
    return etree.tostring(root[0], pretty_print=True)

In [12]:
def get_pmid(doi):
    """Function to retrieve UID crosswalk from pubmed.
       Pass it a doi as a string, starting with the 10..., and it will download the json results."""
#     url = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?ids=" + doi + "&format=json"
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=" + doi + "&format=json"
#    print(url)
    r = requests.get(url)
    results = json.loads(r.text)
    r.close()
    time.sleep(2)
    count = results['esearchresult']['count']
    if count == "0":
        return "ERROR" #"ZERO ERROR"
    elif count == "1":
        ids = results['esearchresult']['idlist']
        if len(ids) != 1:
            return  "ERROR" #"NUMBER OF IDS ERROR"
        else:
            return ids[0]
    else:
#         print(results)
        return  "ERROR" #"MORE THAN 1 RESULT ERROR"
    
def assemble_date(node):
    "takes in a node and extracts values and assembles iso date"
    day = node.xpath('./day/text()')
    assert len(day) == 1
    month = node.xpath('./month/text()')
    assert len(month) == 1
    year = node.xpath('./year/text()')
    assert len(month) == 1
    daytext = day[0].strip()
    monthtext = month[0].strip()
    yeartext = year[0].strip()
    date = '-'.join([yeartext, monthtext, daytext])
    return date

def create_journal(node):
    result = ""
    names = node.xpath('.//name')
    formattednames = []
    for n in names:
        sur = "".join(n.xpath('./surname//text()'))
        gn = "".join(n.xpath('./given-names//text()'))
        formattednames.append(sur + ", " + gn)
    result += ", ".join(formattednames)
    
#     title = " ".join("".join(node.xpath('.//article-title//text()')))
    result += " " + " ".join(("".join(node.xpath('.//article-title//text()'))).split())
    result += " " + " ".join(("".join(node.xpath('.//source//text()'))).split())
    result += " " + " ".join(("".join(node.xpath('.//year//text()'))).split())  + "."
    try:
        result += " " + node.xpath('.//volume//text()')[0]
        try:
            result += ":" + node.xpath('.//fpage//text()')[0] + "-" + node.xpath('.//lpage//text()')[0] + "."
        except:
            result += "."
    except:
        pass
    
    return result

def create_book(node):
    return " ".join(("".join(node.xpath('.//text()'))).split())
#     result = ""
#     names = node.xpath('.//name')
#     formattednames = []
#     for n in names:
#         sur = "".join(n.xpath('./surname//text()'))
#         gn = "".join(n.xpath('./given-names//text()'))
#         formattednames.append(sur + ", " + gn[0].upper() + ".")
#     result += ", ".join(formattednames)
    
    

def assemble_citation(node):
    "accept a single ref element and assemble a single string citation from it."
    reftype = node.xpath('./*/@publication-type')
    if len(reftype) == 0:
        # dump out of lof file of all these
        ismixed = node.xpath('./mixed-citation')
        isnote = node.xpath('./note')
        # isolating these so we can treat them differently if we want, but not right now
        if ismixed:
            return create_book(ismixed[0])
        elif isnote:
            return create_book(isnote[0])
        else:
            grabanything = node.xpath('./*')
            return create_book(grabanything[0])
#             return '' # return "ERROR" # suppressing error that happens ~120 times, not needed 
    elif reftype[0] == 'journal':
        return create_journal(node)
    elif reftype[0] == 'book':
        return create_book(node)
    elif reftype[0] == 'weblink':
        return create_book(node)
    else:
        return create_book(node)


def process_record(path):
#     print(path)
    data = {'sbol:displayId': "", 'doi': ""}
    xml = path.read_text(encoding='utf-8').encode('utf-8')
    root = etree.fromstring(xml)
    ns = root.nsmap

    # get 'sbol:displayId' from article id
    #article_id = root.xpath('//article/@id', namespaces = ns)
    article_id = str(pathlib.Path(str(path.stem)))
    print(article_id)
    #assert len(article_id) == 1
    #data['sbol:displayId'] = article_id[0]
    data['sbol:displayId'] = article_id
    print(article_id)
    if article_id in chemicals:   
         print('chemical found')
         data['chemicals'] = chemicals[article_id]
    if article_id in species:   
         print('specie found')
         data['species'] = species[article_id]
    if article_id in strains:   
         print('strain found')
         data['strains'] = strains[article_id]
#     print(chemicals)
    
    # get doi
    doi = root.xpath('//article-id[@pub-id-type = "doi"]/text()')
    assert len(doi) == 1
    data['doi_formatted'] = "https://pubs.acs.org/doi/" + doi[0]
    data['doi'] = doi[0]
    pmdoi = get_pmid(data['doi'])
    data['pmid'] = pmdoi
    
    # get article type
    article_type = root.xpath('//article/@article-type')
    if len(article_type) == 1:
        data['article_type'] = article_type[0]
    else:
        data['article_type'] = "Research Article"
        
    # get journal title
    jtitle = root.xpath('//journal-title-group/journal-title//text()')
    if len(jtitle) == 1:
        data['jounal_title'] = jtitle[0]
    else:
        print("warning missing title")
        data['jounal_title'] = "ACS Synthetic Biology"
        
    # get journal publisher
    jpublisher = root.xpath('//publisher/publisher-name//text()')
    if len(jpublisher) == 1:
        data['jounal_publisher'] = jpublisher[0]
    else:
        print("warning missing publisher")
        data['jounal_publisher'] = "American Chemical Society"
        
    # get article title
    
    atitle = root.xpath('//title-group/article-title//text()')
    data['article-title'] = " ".join(" ".join(atitle).split())
    
    # get authors
    authors = root.xpath('//contrib-group/contrib[@contrib-type = "author"]')
    authornames = []
    for a in authors:
        surname = " ".join(a.xpath('./name/surname/text()')).replace('\n', ' ')
        givennames = " ".join(a.xpath('./name/given-names/text()')).replace('\n', ' ')
        name = givennames + " " + surname
        authornames.append(name)
    data['authors'] = authornames
    
    # get created date
    
    creatednode = root.xpath('//history/date[@date-type="issue-pub"]')
    
    assert len(creatednode) <= 1
    if len(creatednode) == 1:
        data['created_date'] = assemble_date(creatednode[0])
    else:
        data['created_date'] = "ERROR"#"Missing"
        print(path)

    # get date submitted
    
    submittednode = root.xpath('//history/date[@date-type = "received"]')
    assert len(submittednode) == 1
    data['submitted_date'] = assemble_date(submittednode[0])
    
    # get copyright date
    
    copyrightdatenode = root.xpath('//permissions/copyright-year/text()')
    assert len(copyrightdatenode) == 1
    data['copyrightyear'] = copyrightdatenode[0]
    
    # get accepted date
    
    accepteddatenode = root.xpath('//history/date[@date-type ="just-accepted"]')
    if len(accepteddatenode) == 1:
        data['accepteddate'] = assemble_date(accepteddatenode[0])
    else:
        data['accepteddate'] = "ERROR" # "Missing"
    
    # get copyright statement
    
    copyrightstatement = root.xpath('//permissions/copyright-statement/text()')
    if len(copyrightstatement) == 1:
        data['copyrightstatement'] = copyrightstatement[0]
    else:
        data['copyrightstatement'] = "ERROR" # "Missing"
    
    # get rights holder
    
    rightsholder = root.xpath('//permissions/copyright-holder/text()')
    assert len(rightsholder) == 1
    data['rightsholder'] = rightsholder[0]
    
    # get abstract
    
    abstract = root.xpath('//abstract[not(@*)]//text()')
    spotlight = root.xpath('//abstract[@abstract-type = "spotlight-teaser"]//text()')
    if len(abstract) > 0:
        data['abstract'] = "".join(abstract).strip().replace('\n', ' ')
    elif len(spotlight) > 0:
        data['abstract'] = "".join(spotlight).strip().replace('\n', ' ')
    elif len(root.xpath('//abstract')) == 0:
        data['abstract'] = "ERROR" # "Missing"
    else:
        assert False
    
    data['description'] = data['abstract']
    
    # get keywords
    
    kwds = root.xpath('//kwd-group/kwd')
    keywords = []
    for k in kwds:
        k = "".join(k.xpath('.//text()'))
        keywords.append(k)
    data['keywords'] = keywords
    #print(path.stem)
    #data['keywords'] += load_ner_mentions(path.stem)
    
    #data['keywords'] = [x.strip() for x in list(set(data['keywords']))]
    # get NER keywords
    # in progress
    # do function here..... but it'll be
    # data['keywords'] += result list from the function...
    
#     species = []
#     data['species'] = load_species
    
#     chemical = []
#     data['chemical'] = load_chemicals
    
    # assemble citations
    
    refs = root.xpath('//ref-list/ref')
    formattedrefs = []
    for r in refs:
        citation = assemble_citation(r)
        if citation == '':
            with open('citationlog.csv', 'a') as log:
                out = csv.writer(log)
                out.writerow([path, " ".join([t.strip() for t in r.xpath('.//text()')])])
            
        formattedrefs.append(citation)
    
    data['refs'] = formattedrefs
    
    return data

In [13]:
data = []

for f in originalfiles:
    xmlfname = f.parent / pathlib.Path(str(f.stem) + '-RDF.xml')
    print(xmlfname)
    # print(chemical)
    j = process_record(f)
    xml = create_new_xml(j)
    xmlfname.write_bytes(xml)

Data\Data\sb200002m\sb200002m-RDF.xml
sb200002m
sb200002m
chemical found
specie found
adding chemical
adding species
Data\Data\sb200003v\sb200003v-RDF.xml
sb200003v
sb200003v
Data\Data\sb200005w\sb200005w-RDF.xml
sb200005w
sb200005w
Data\Data\sb2000066\sb2000066-RDF.xml
sb2000066
sb2000066
Data\Data\sb200007r\sb200007r-RDF.xml
sb200007r
sb200007r
Data\Data\sb200008j\sb200008j-RDF.xml
sb200008j
sb200008j
Data\Data\sb200010w\sb200010w-RDF.xml
sb200010w
sb200010w
chemical found
specie found
adding chemical
adding species
Data\Data\sb200015u\sb200015u-RDF.xml
sb200015u
sb200015u
chemical found
specie found
adding chemical
adding species
Data\Data\sb200017p\sb200017p-RDF.xml
sb200017p
sb200017p
chemical found
specie found
adding chemical
adding species
Data\Data\sb200018e\sb200018e-RDF.xml
sb200018e
sb200018e
chemical found
specie found
adding chemical
adding species
Data\Data\sb200019x\sb200019x-RDF.xml
sb200019x
sb200019x
chemical found
specie found
adding chemical
adding species
Data\Dat

adding chemical
adding species
Data\Data\sb3001003\sb3001003-RDF.xml
sb3001003
sb3001003
chemical found
specie found
strain found
adding chemical
adding species
adding strain
obo:NCBITaxon_477245
Data\Data\sb300101g\sb300101g-RDF.xml
sb300101g
sb300101g
Data\Data\sb300102k\sb300102k-RDF.xml
sb300102k
sb300102k
chemical found
specie found
adding chemical
adding species
Data\Data\sb300104f\sb300104f-RDF.xml
sb300104f
sb300104f
Data\Data\sb300105b\sb300105b-RDF.xml
sb300105b
sb300105b
Data\Data\sb3001062\sb3001062-RDF.xml
sb3001062
sb3001062
Data\Data\sb300107h\sb300107h-RDF.xml
sb300107h
sb300107h
Data\Data\sb3001084\sb3001084-RDF.xml
sb3001084
sb3001084
chemical found
specie found
adding chemical
adding species
Data\Data\sb300109d\sb300109d-RDF.xml
sb300109d
sb300109d
Data\Data\sb300110b\sb300110b-RDF.xml
sb300110b
sb300110b
chemical found
specie found
adding chemical
adding species
Data\Data\sb3001112\sb3001112-RDF.xml
sb3001112
sb3001112
Data\Data\sb300114d\sb300114d-RDF.xml
sb300114d

adding chemical
adding species
Data\Data\sb400082j\sb400082j-RDF.xml
sb400082j
sb400082j
Data\Data\sb4000838\sb4000838-RDF.xml
sb4000838
sb4000838
Data\Data\sb400086p\sb400086p-RDF.xml
sb400086p
sb400086p
chemical found
specie found
strain found
adding chemical
adding species
adding strain
obo:NCBITaxon_511145
Data\Data\sb400087e\sb400087e-RDF.xml
sb400087e
sb400087e
Data\Data\sb400089c\sb400089c-RDF.xml
sb400089c
sb400089c
Data\Data\sb400090s\sb400090s-RDF.xml
sb400090s
sb400090s
Data\Data\sb400091p\sb400091p-RDF.xml
sb400091p
sb400091p
Data\Data\sb400092e\sb400092e-RDF.xml
sb400092e
sb400092e
Data\Data\sb400093x\sb400093x-RDF.xml
sb400093x
sb400093x
Data\Data\sb400094c\sb400094c-RDF.xml
sb400094c
sb400094c
chemical found
adding chemical
Data\Data\sb4000959\sb4000959-RDF.xml
sb4000959
sb4000959
chemical found
specie found
adding chemical
adding species
Data\Data\sb4000965\sb4000965-RDF.xml
sb4000965
sb4000965
chemical found
specie found
adding chemical
adding species
Data\Data\sb40009

Data\Data\sb500003x\sb500003x-RDF.xml
sb500003x
sb500003x
chemical found
specie found
adding chemical
adding species
Data\Data\sb5000059\sb5000059-RDF.xml
sb5000059
sb5000059
chemical found
specie found
adding chemical
adding species
Data\Data\sb5000077\sb5000077-RDF.xml
sb5000077
sb5000077
Data\Data\sb5000115\sb5000115-RDF.xml
sb5000115
sb5000115
chemical found
specie found
strain found
adding chemical
adding species
adding strain
obo:NCBITaxon_83333
Data\Data\sb5000143\sb5000143-RDF.xml
sb5000143
sb5000143
Data\Data\sb500018f\sb500018f-RDF.xml
sb500018f
sb500018f
Data\Data\sb500019b\sb500019b-RDF.xml
sb500019b
sb500019b
chemical found
specie found
adding chemical
adding species
Data\Data\sb500020g\sb500020g-RDF.xml
sb500020g
sb500020g
Data\Data\sb500023f\sb500023f-RDF.xml
sb500023f
sb500023f
Data\Data\sb500024b\sb500024b-RDF.xml
sb500024b
sb500024b
chemical found
specie found
adding chemical
adding species
Data\Data\sb5000252\sb5000252-RDF.xml
sb5000252
sb5000252
Data\Data\sb500029n\

Data\Data\sb500272d\sb500272d-RDF.xml
sb500272d
sb500272d
Data\Data\sb500273n\sb500273n-RDF.xml
sb500273n
sb500273n
chemical found
specie found
adding chemical
adding species
Data\Data\sb500278z\sb500278z-RDF.xml
sb500278z
sb500278z
Data\Data\sb500278z\sb500278z.xml
Data\Data\sb500279w\sb500279w-RDF.xml
sb500279w
sb500279w
Data\Data\sb500280q\sb500280q-RDF.xml
sb500280q
sb500280q
Data\Data\sb500282v\sb500282v-RDF.xml
sb500282v
sb500282v
chemical found
specie found
Data\Data\sb500282v\sb500282v.xml
adding chemical
adding species
Data\Data\sb5002856\sb5002856-RDF.xml
sb5002856
sb5002856
chemical found
specie found
adding chemical
adding species
Data\Data\sb500286r\sb500286r-RDF.xml
sb500286r
sb500286r
Data\Data\sb500286r\sb500286r.xml
Data\Data\sb500287j\sb500287j-RDF.xml
sb500287j
sb500287j
Data\Data\sb5002888\sb5002888-RDF.xml
sb5002888
sb5002888
Data\Data\sb5002906\sb5002906-RDF.xml
sb5002906
sb5002906
Data\Data\sb500291r\sb500291r-RDF.xml
sb500291r
sb500291r
chemical found
specie fou

...

Data\Data\sb8b00379\sb8b00379.xml
Data\Data\sb8b00380\sb8b00380-RDF.xml
sb8b00380
sb8b00380
chemical found
specie found
Data\Data\sb8b00380\sb8b00380.xml
adding chemical
adding species
Data\Data\sb8b00381\sb8b00381-RDF.xml
sb8b00381
sb8b00381
chemical found
specie found
Data\Data\sb8b00381\sb8b00381.xml
adding chemical
adding species
Data\Data\sb8b00383\sb8b00383-RDF.xml
sb8b00383
sb8b00383
Data\Data\sb8b00383\sb8b00383.xml
Data\Data\sb8b00385\sb8b00385-RDF.xml
sb8b00385
sb8b00385
chemical found
specie found
Data\Data\sb8b00385\sb8b00385.xml
adding chemical
adding species
Data\Data\sb8b00386\sb8b00386-RDF.xml
sb8b00386
sb8b00386
chemical found
specie found
Data\Data\sb8b00386\sb8b00386.xml
adding chemical
adding species
Data\Data\sb8b00388\sb8b00388-RDF.xml
sb8b00388
sb8b00388
chemical found
specie found
Data\Data\sb8b00388\sb8b00388.xml
adding chemical
adding species
Data\Data\sb8b00392\sb8b00392-RDF.xml
sb8b00392
sb8b00392
Data\Data\sb8b00392\sb8b00392.xml
Data\Data\sb8b00393\sb8b003

Data\Data\sb8b00489\sb8b00489.xml
adding chemical
adding species
Data\Data\sb8b00493\sb8b00493-RDF.xml
sb8b00493
sb8b00493
Data\Data\sb8b00493\sb8b00493.xml
Data\Data\sb8b00496\sb8b00496-RDF.xml
sb8b00496
sb8b00496
Data\Data\sb8b00496\sb8b00496.xml
Data\Data\sb8b00498\sb8b00498-RDF.xml
sb8b00498
sb8b00498
chemical found
specie found
Data\Data\sb8b00498\sb8b00498.xml
adding chemical
adding species
Data\Data\sb8b00499\sb8b00499-RDF.xml
sb8b00499
sb8b00499
Data\Data\sb8b00499\sb8b00499.xml
Data\Data\sb8b00501\sb8b00501-RDF.xml
sb8b00501
sb8b00501
chemical found
specie found
Data\Data\sb8b00501\sb8b00501.xml
adding chemical
adding species
Data\Data\sb8b00503\sb8b00503-RDF.xml
sb8b00503
sb8b00503
specie found
strain found
Data\Data\sb8b00503\sb8b00503.xml
adding species
adding strain
obo:NCBITaxon_9615
Data\Data\sb8b00505\sb8b00505-RDF.xml
sb8b00505
sb8b00505
Data\Data\sb8b00505\sb8b00505.xml
Data\Data\sb8b00507\sb8b00507-RDF.xml
sb8b00507
sb8b00507
Data\Data\sb8b00507\sb8b00507.xml
Data\Da

Data\Data\sb9b00055\sb9b00055.xml
Data\Data\sb9b00058\sb9b00058-RDF.xml
sb9b00058
sb9b00058
Data\Data\sb9b00058\sb9b00058.xml
Data\Data\sb9b00059\sb9b00059-RDF.xml
sb9b00059
sb9b00059
Data\Data\sb9b00059\sb9b00059.xml
Data\Data\sb9b00060\sb9b00060-RDF.xml
sb9b00060
sb9b00060
chemical found
specie found
strain found
Data\Data\sb9b00060\sb9b00060.xml
adding chemical
adding species
adding strain
obo:NCBITaxon_511693
Data\Data\sb9b00061\sb9b00061-RDF.xml
sb9b00061
sb9b00061
chemical found
specie found
Data\Data\sb9b00061\sb9b00061.xml
adding chemical
adding species
Data\Data\sb9b00062\sb9b00062-RDF.xml
sb9b00062
sb9b00062
chemical found
specie found
Data\Data\sb9b00062\sb9b00062.xml
adding chemical
adding species
Data\Data\sb9b00063\sb9b00063-RDF.xml
sb9b00063
sb9b00063
chemical found
specie found
Data\Data\sb9b00063\sb9b00063.xml
adding chemical
adding species
Data\Data\sb9b00066\sb9b00066-RDF.xml
sb9b00066
sb9b00066
Data\Data\sb9b00066\sb9b00066.xml
Data\Data\sb9b00068\sb9b00068-RDF.xml


Data\Data\sb9b00169\sb9b00169.xml
adding chemical
adding species
Data\Data\sb9b00171\sb9b00171-RDF.xml
sb9b00171
sb9b00171
chemical found
specie found
Data\Data\sb9b00171\sb9b00171.xml
adding chemical
adding species
Data\Data\sb9b00172\sb9b00172-RDF.xml
sb9b00172
sb9b00172
Data\Data\sb9b00172\sb9b00172.xml
Data\Data\sb9b00174\sb9b00174-RDF.xml
sb9b00174
sb9b00174
Data\Data\sb9b00174\sb9b00174.xml
Data\Data\sb9b00175\sb9b00175-RDF.xml
sb9b00175
sb9b00175
chemical found
specie found
Data\Data\sb9b00175\sb9b00175.xml
adding chemical
adding species
Data\Data\sb9b00176\sb9b00176-RDF.xml
sb9b00176
sb9b00176
chemical found
specie found
Data\Data\sb9b00176\sb9b00176.xml
adding chemical
adding species
Data\Data\sb9b00177\sb9b00177-RDF.xml
sb9b00177
sb9b00177
Data\Data\sb9b00177\sb9b00177.xml
Data\Data\sb9b00178\sb9b00178-RDF.xml
sb9b00178
sb9b00178
chemical found
specie found
Data\Data\sb9b00178\sb9b00178.xml
adding chemical
adding species
Data\Data\sb9b00179\sb9b00179-RDF.xml
sb9b00179
sb9b001

adding chemical
adding species
Data\Data\sbv001i001\sb2000287\sb2000287-RDF.xml
sb2000287
sb2000287
Data\Data\sbv001i001\sb300001c\sb300001c-RDF.xml
sb300001c
sb300001c
Data\Data\sbv001i001\sb3000047\sb3000047-RDF.xml
sb3000047
sb3000047
Data\NER Data\sb2000275\sb2000275-RDF.xml
sb2000275
sb2000275
chemical found
specie found
adding chemical
adding species
Data\NER Data\sb3001084\sb3001084-RDF.xml
sb3001084
sb3001084
chemical found
specie found
adding chemical
adding species
Data\NER Data\sb400051t\sb400051t-RDF.xml
sb400051t
sb400051t
chemical found
specie found
adding chemical
adding species
Data\NER Data\sb5b00012\sb5b00012-RDF.xml
sb5b00012
sb5b00012
chemical found
specie found
adding chemical
adding species
Data\NER Data\sb6b00027\sb6b00027-RDF.xml
sb6b00027
sb6b00027
chemical found
specie found
Data\NER Data\sb6b00027\sb6b00027.xml
adding chemical
adding species
Data\NER Data\sb6b00135\sb6b00135-RDF.xml
sb6b00135
sb6b00135
chemical found
specie found
Data\NER Data\sb6b00135\sb6b0

In [14]:
allfiles = list(pathlib.Path('data_folders/').glob('**/*.xml'))
newfiles = [f for f in xmlfiles if f.stem.endswith('-RDF')]

In [15]:
selfclosing = re.compile(r'/>')
findamp = re.compile(r'&#169;')

for f in newfiles:
    text = f.read_text()
    text = re.sub(selfclosing, ' />', text)
    text = re.sub(findamp, '&amp;', text)
    try:
        tree = etree.fromstring(text)
        f.write_text(text)
    except:
        print("failure", f)